# Project Code

In [1]:
#import libraries
import numpy as np
import pandas as pd
import re
from textblob import TextBlob as tb ##TextBlob object will allow for quick sentiment analysis
from emo_unicode import EMO_UNICODE #A set of dictionaries of emoticons : their text represenstations
from emo_unicode import UNICODE_EMO ## courtesy of NeelShah18 on GitHub
                                  

ModuleNotFoundError: No module named 'emo_unicode'

In [ ]:
#read tweet information into dataframe
tweets = pd.read_csv('tweets.csv')

In [ ]:
#replace emojis with their text representation
##this method is courtesy of kaggle.com user SRK

UNICODE_EMO = {v: k for k, v in EMO_UNICODE.items()}

def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
    return text

In [ ]:
#create series objects of the polarity and subjectivity of the tweets
polarity = tweets.text.map(lambda x: tb(x).sentiment.polarity)
subjectivity = tweets.text.map(lambda x: tb(x).sentiment.subjectivity)

In [ ]:
#add new columns to tweets df corresponding to the new info
tweets['polarity'] = polarity
tweets['subjectivity'] = subjectivity
tweets.head()

In [ ]:
#print stuff about the dataframe
tweets.describe()

In [ ]:
#print more stuff about the dataframe
tweets.info()

In [ ]:
#make new column that shows year
##we will use this column to group tweets together when making our comparisons
tweets['created_at'] = pd.to_datetime(tweets['created_at'])
tweets['year'] = tweets['created_at'].dt.year.astype('Int64')
tweets.head()

In [ ]:
#create the subsets for each topic
russia = tweets['text'].str.contains('russia|Russia|moscow|Moscow|putin|Putin')
iran = tweets['text'].str.contains('iran|Iran|tehran|Tehran|Nuclear Deal|Rouhani')
tweets[iran]

In [ ]:
tweets[russia]['year'].value_counts().sort_index()